<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек-и-чтение-данных" data-toc-modified-id="Импорт-библиотек-и-чтение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек и чтение данных</a></span></li><li><span><a href="#Создание-корпуса" data-toc-modified-id="Создание-корпуса-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Создание корпуса</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Мешок-слов" data-toc-modified-id="Мешок-слов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Мешок слов</a></span><ul class="toc-item"><li><span><a href="#Обучение-без-стопслов" data-toc-modified-id="Обучение-без-стопслов-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Обучение без стопслов</a></span></li><li><span><a href="#Обучение-со-стопсловами" data-toc-modified-id="Обучение-со-стопсловами-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Обучение со стопсловами</a></span></li></ul></li><li><span><a href="#Мешок-слов-с-n-граммами" data-toc-modified-id="Мешок-слов-с-n-граммами-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Мешок слов с n-граммами</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>TF-IDF</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>BERT</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп» (c BERT)

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт библиотек и чтение данных

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from tqdm import notebook 

import torch
import transformers
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score
from sklearn.utils import shuffle

Загрузим дополнительные компоненты библиотеки `nltk`, такие как **stopwords** и **punkt**, необходимые для дальнешей работы.

In [2]:
nltk.download('stopwords')
nltk.download('punkt') 

stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\femt0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\femt0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Прочитаем данные из файла в переменную типа DataFrame

In [3]:
data = pd.read_csv('./datasets/toxic_comments.csv')

data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Изучим баланс классов:

In [5]:
balance = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
print(
    f'Отношение отрицательного класса к положительному составляет {balance:.1f}:1')

Отношение отрицательного класса к положительному составляет 8.8:1


In [6]:
data['toxic'].value_counts()[0]

143346

**Промежуточный вывод:**

Данные не требуют дополнительной подготовки к дальнешей работе - пропуски отсутствуют, типы данных соответствуют.

### Создание корпуса

Приведём текст сообщений единообразной форме, т.е. сформируем корпус.

в том числе приведём слова к их лемме, и очистим от "мусорных" слов и символов.

1. Токенизируем текст с используем библиотеки `nltk.tokenize.TweetTokenizer`, как более подходящее под тип текста (интернет-общение):

In [7]:
%%time

tknzr = TweetTokenizer()

tokenized = data['text'].str.lower().apply(tknzr.tokenize)

Wall time: 46.8 s


2. Очистим текст, оставив только буквы латинского алфавита с использованием регулярных выражений:

In [8]:
def clean(text):
    clean = re.sub(r'[^a-zA-Z\' ]', ' ', text)
    return " ".join(clean.split())

In [9]:
clean_corpus = []
for text in notebook.tqdm(tokenized.values):
    clean_corpus.append(clean(" ".join(text)))

  0%|          | 0/159571 [00:00<?, ?it/s]

3. Воспользуемся лемматизатором `WordNetLemmatizer` из той же библиотеки `nltk`:

In [10]:
lemmatized = []
wnl = WordNetLemmatizer()

for text in notebook.tqdm(clean_corpus):
    lemmatized.append(" ".join([wnl.lemmatize(word) for word in text.split()]))

  0%|          | 0/159571 [00:00<?, ?it/s]

4. Добавим полученный результат обратно в таблицу в виде нового столбца:

In [11]:
data['lemmatized_text'] = lemmatized

In [12]:
data.head(5)

,text,toxic,lemmatized_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i'm seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i'm really not trying to edit war it's...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can't make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


Выделим признаки в переменные **X** и **y**, и разделим данные на тренировочную и валидационную выборку:

In [13]:
X = data['lemmatized_text'].copy()
y = data['toxic'].copy()

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=13)

В связи с дисбалансом классов опробуем вариант выравнивания этого баланса методом "**upsample**" (увеличение выборки). На данных выборках будем обучать только модели, менее склонные к переобучению, т.е. линейные.

In [15]:
def upsample(X, y, up_coeff):
    X_zeros = X[y == 0]
    X_ones = X[y == 1]
    y_zeros = y[y == 0]
    y_ones = y[y == 1]
    
    X_upsampled, y_upsampled = shuffle(
        pd.concat([X_zeros] + [X_ones] * up_coeff),
        pd.concat([y_zeros] + [y_ones] * up_coeff),
        random_state=13)
    
    return X_upsampled, y_upsampled

Создадим переменные, использующие данный метод.

In [16]:
X_train_up, y_train_up = upsample(X_train, y_train, 8)

print(f'{"Классы": ^12}{"0": ^10}{"1": ^12}')
print(f'{"imbalanced": ^12}{y_train.value_counts()[0]: ^10}{y_train.value_counts()[1]: ^12}')
print(f'{"balanced": ^12}{y_train_up.value_counts()[0]: ^10}{y_train_up.value_counts()[1]: ^12}')

   Классы       0          1      
 imbalanced   114593     13063    
  balanced    114593     104504   


**Вывод**

Текстовые данные приведены к единообразной форме, и готовы к обучению моделей.

## Обучение

Результаты работы всех моделей будут собраны в новый *DataFrame* **results**

In [17]:
columns = ['method', 'model_name', 'F1', 'fit_time', 'predict_time']

results = pd.DataFrame(columns=columns)

Обучение будем производить на одинаковых моделях, в независимости от способа подготовки данных, а именно `LogisticRegression()`, `Perceptron()`,`DecisionTreeClassifier()`:

In [18]:
models = [
    LogisticRegression(max_iter=1500),
    Perceptron(),
    DecisionTreeClassifier(max_depth=100, random_state=13)
]

Добавим функцию, заполняющая таблицу с результатами:

In [19]:
def record(model, method, result_df=results,
           X=X_train,
           y=y_train,
           X_tru=X_test,
           y_tru=y_test):
    model_name = model.__class__.__name__
    
    time_start = datetime.now()
    model.fit(X, y)
    time_mid = datetime.now()
    y_pred = model.predict(X_tru)
    f1_ = f1_score(y_tru, y_pred)
    time_end = datetime.now()
    result_df.loc[result_df.shape[0]] = [method, model_name, f1_,
                               (time_mid - time_start).total_seconds(),
                               (time_end - time_mid).total_seconds()]
    print(f'Model {model_name}, F1 = {f1_}')
    #return model, y_pred

Добавим в таблицу значение **baseline**, для оценки адекватности моделей. В качестве таковой используем `DummyClassifier()`

In [20]:
baseline = DummyClassifier(strategy="constant", random_state=13, constant=1)

record(model=baseline,
       method='baseline',
       X=X_train, y=y_train,
       X_tru=X_test, y_tru=y_test)

Model DummyClassifier, F1 = 0.18028907831342475


### Мешок слов

Преобразуем текст в векторный формат методом **"мешок слов"**

#### Обучение без стопслов

Создадим "мешок слов" с удалением "**стопслов**", загруженных раннее:

In [21]:
count_vect = CountVectorizer(stop_words=stopwords) 
bow_train = count_vect.fit_transform(X_train.values)
bow_test = count_vect.transform(X_test.values)

print('Размер мешка слов с учётом стоп-слов:')
print(f'train: {bow_train.shape[0]} на {bow_train.shape[1]}')
print(f'test: {bow_test.shape[0]} на {bow_test.shape[1]}')

Размер мешка слов с учётом стоп-слов:
train: 127656 на 139030
test: 31915 на 139030


Произведём обучение с использованием вышеуказанных моделей (`LogisticRegression()`, `Perceptron()`,`DecisionTreeClassifier()`)

In [22]:
for model in notebook.tqdm(models):
    record(model=model,
           method='BOW w/ stopwords',
           X=bow_train,
           X_tru=bow_test)

  0%|          | 0/3 [00:00<?, ?it/s]

Model LogisticRegression, F1 = 0.7526278906797479
Model Perceptron, F1 = 0.7004073958006894
Model DecisionTreeClassifier, F1 = 0.7101300479123887


#### Обучение со стопсловами

Также проверим аналогичный подход, не удаляя "*стопслова*", как предположительно необходимые для определения "*токсичности*"

In [23]:
count_vect = CountVectorizer() 
bow_train_2 = count_vect.fit_transform(X_train.values)
bow_test_2 = count_vect.transform(X_test.values)

print('Размер мешка слов без учёта стоп-слов:')
print(f'train: {bow_train_2.shape[0]} на {bow_train_2.shape[1]}')
print(f'test: {bow_test_2.shape[0]} на {bow_test_2.shape[1]}')

Размер мешка слов без учёта стоп-слов:
train: 127656 на 139174
test: 31915 на 139174


Аналогичным образом проверим результат для обучения в полном объёме данных, с сохранёнными стопсловами:

In [24]:
for model in notebook.tqdm(models):
    record(model=model,
           method='BOW w/o stopwords',
           X=bow_train_2,
           X_tru=bow_test_2)

  0%|          | 0/3 [00:00<?, ?it/s]

Model LogisticRegression, F1 = 0.771295818275684
Model Perceptron, F1 = 0.7283086439850237
Model DecisionTreeClassifier, F1 = 0.715045462343455


### Мешок слов с n-граммами


Создадим мешок слов образованием словосочетаний (**N-грамм**), по основанию 1 и 2.

In [25]:
n_gramm_count_vect = CountVectorizer(ngram_range=(1,2)) 
n_gramm_train = n_gramm_count_vect.fit_transform(X_train.values)
n_gramm_test = n_gramm_count_vect.transform(X_test.values)


print('Размер мешка слов c N-граммами:')
print(f'train: {n_gramm_train.shape[0]} на {n_gramm_train.shape[1]}')
print(f'test: {n_gramm_test.shape[0]} на {n_gramm_test.shape[1]}')

Размер мешка слов c N-граммами:
train: 127656 на 1892362
test: 31915 на 1892362


Добавим результаты обучения на основе мешка слов с n-граммами (в данном случае - униграммы и биграммы):

In [26]:
for model in notebook.tqdm(models):
    record(model=model,
           method='N_gram',
           X=n_gramm_train,
           X_tru=n_gramm_test)

  0%|          | 0/3 [00:00<?, ?it/s]

Model LogisticRegression, F1 = 0.7753534648280677
Model Perceptron, F1 = 0.7640521823159929
Model DecisionTreeClassifier, F1 = 0.7144089732528042


Также проверим мешок слов с n-граммами на отбалансированных данных:

In [27]:
n_gramm_train_up = n_gramm_count_vect.fit_transform(X_train_up.values)
n_gramm_test_up = n_gramm_count_vect.transform(X_test.values)


print('Размер мешка слов c N-граммами:')
print(f'train: {n_gramm_train_up.shape[0]} на {n_gramm_train_up.shape[1]}')
print(f'test: {n_gramm_test_up.shape[0]} на {n_gramm_test_up.shape[1]}')

Размер мешка слов c N-граммами:
train: 219097 на 1892362
test: 31915 на 1892362


In [28]:
for model in notebook.tqdm([LogisticRegression(max_iter=1500), Perceptron()]):
    record(model=model,
           method='N_gram_upsampled',
           X=n_gramm_train_up, y=y_train_up,
           X_tru=n_gramm_test_up)

  0%|          | 0/2 [00:00<?, ?it/s]

Model LogisticRegression, F1 = 0.7957263594323075
Model Perceptron, F1 = 0.7707850281550184


### TF-IDF

Создадим матрицы частоты употребления слов **TF-IDF** с использованием части библиотеки `sklearn` под названием `TfidfVectorizer`

In [29]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
X_train_tf = count_tf_idf.fit_transform(X_train.values.astype('U'))
X_test_tf = count_tf_idf.transform(X_test.values.astype('U'))


print('Размер матриц TF-IDF:')
print(f'train: {X_train_tf.shape[0]} на {X_train_tf.shape[1]}')
print(f'test: {X_test_tf.shape[0]} на {X_test_tf.shape[1]}')

Размер матриц TF-IDF:
train: 127656 на 139030
test: 31915 на 139030


Аналогичным образом добавим результаты для моделей с **TF-IDF**:

In [30]:
for model in notebook.tqdm(models):
    record(model=model,
           method='TF_IDF',
           X=X_train_tf,
           X_tru=X_test_tf)

  0%|          | 0/3 [00:00<?, ?it/s]

Model LogisticRegression, F1 = 0.7270296084049666
Model Perceptron, F1 = 0.7098674521354933
Model DecisionTreeClassifier, F1 = 0.7237206085753803


Также проверим данный метод на отбалансированных данных:

In [31]:
X_train_tf_up = count_tf_idf.fit_transform(X_train_up.values.astype('U'))
X_test_tf_up = count_tf_idf.transform(X_test.values.astype('U'))

print('Размер матриц TF-IDF:')
print(f'train: {X_train_tf_up.shape[0]} на {X_train_tf_up.shape[1]}')
print(f'test: {X_test_tf_up.shape[0]} на {X_test_tf_up.shape[1]}')

Размер матриц TF-IDF:
train: 219097 на 139030
test: 31915 на 139030


In [32]:
for model in notebook.tqdm([LogisticRegression(max_iter=1500), Perceptron()]):
    record(model=model,
           method='TF_IDF_upsampled',
           X=X_train_tf_up, y=y_train_up,
           X_tru=X_test_tf_up)

  0%|          | 0/2 [00:00<?, ?it/s]

Model LogisticRegression, F1 = 0.7504584567640006
Model Perceptron, F1 = 0.7001044932079414


### BERT

Воспользуемся библиотекой **BERT** (*Bidirectional Encoder Representations from Transformers*) за авторством Google, для создания векторного представления корпуса текстов.

Воспользуемся предобученной моделью `bert-base-uncased` для наших целей.

In [33]:
tokenizer = transformers.BertTokenizer(vocab_file='./datasets/vocab.txt')

model = transformers.BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Проверим устройство на наличие CUDA-совместимых ускорителей, и перенесём туда нашу модель BERT:

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda:0


Токенизируем текст методами BERT (преобразуем текст в номера токенов словаря). Максимальную длину устанавливаем равную 512, при больших значениях предобученная модель останавливается с ошибкой:

In [35]:
tokenized = data['lemmatized_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,
                                                                      max_length=512,
                                                                      truncation=True)))

Выравниваем длины исходных текстов в корпусе, и создаем маску:

In [36]:
max_len = tokenized.str.len().max()

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

padded.shape

(159571, 512)

Создаём векторное представление (*embeddings*)

In [37]:
batch_size = 150
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

        
        if (i == padded.shape[0] // batch_size - 1) and (padded.shape[0] % batch_size != 0):
            #last piece of "cake"
            last_piece = padded.shape[0] % batch_size
            batch = torch.LongTensor(
                padded[batch_size*(i+1):(batch_size*(i+1)+last_piece)]
            )
            attention_mask_batch = torch.LongTensor(
                attention_mask[batch_size*(i+1):(batch_size*(i+1)+last_piece)]
            )
            with torch.no_grad():
                batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))
            embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
            
            del batch
            del attention_mask_batch
            del batch_embeddings

  0%|          | 0/1063 [00:00<?, ?it/s]

Объединяем результат в один список, и разделяем его в тренировочную и тестовую выборки.

In [38]:
features = np.concatenate(embeddings)

X_train, X_test, y_train, y_test = train_test_split(features, data['toxic'], test_size=0.25, random_state=13)
X_train_up, y_train_up = upsample(pd.DataFrame(X_train, index=y_train.index), y_train, 8)

print('Размер матриц эмбеддингов:')
print(f'train: {X_train.shape[0]} на {X_train.shape[1]}')
print(f'test: {X_test.shape[0]} на {X_test.shape[1]}')

Размер матриц эмбеддингов:
train: 119678 на 768
test: 39893 на 768


Также обучаем модели и получаем результаты метрики **F1**:

In [39]:
for model in notebook.tqdm([LogisticRegression(max_iter=1500), Perceptron()]):
    record(model=model,
           method='BERT',
           X=X_train, y=y_train,
           X_tru=X_test, y_tru=y_test)

  0%|          | 0/2 [00:00<?, ?it/s]

Model LogisticRegression, F1 = 0.7129381951288187
Model Perceptron, F1 = 0.5544539116963594


In [40]:
for model in notebook.tqdm([LogisticRegression(max_iter=1500), Perceptron()]):
    record(model=model,
           method='BERT_upsampled',
           X=X_train_up, y=y_train_up,
           X_tru=X_test, y_tru=y_test)

  0%|          | 0/2 [00:00<?, ?it/s]

Model LogisticRegression, F1 = 0.6384159881569209
Model Perceptron, F1 = 0.5983052298211076


**Вывод**

Все модели обучены, результаты собраны в одну таблицу, готовую к финальному анализу.

## Выводы

Рассмотрим полученную таблицу с результатами:

In [41]:
results.set_index(['method', 'model_name']).style.background_gradient(cmap='GnBu')

1. Все модели проходят проверку на адекватность: метрика F1 во много раз больше baseline метрики равной **0.18**;
2. Удаление стопслов из корпуса сказалось на результате в сторону ухудшения, предположительно, в случае оценки токсичности - они имеют большее значение, чем ожидалось;
3. Наглядно видно, что **BERT** в данном случае показывает наихудщие результаты, не считая времени необходимого на перевод текста в векторные представления (*embeddings*). Основной причиной, по видимому, является использование предобученной модели `bert-base-uncased`, которая не учитывает специфику наших исходных данных. Также модели BERT балансировка класса методом увеличения выборки (upsample) не приносит должного результата

Выберем из таблицы результатов только удовлетворяющие поставленному условию - метрика ${F1\geq0.75}$

In [42]:
results.query('F1 > 0.75').style.background_gradient(cmap='GnBu')

,method,model_name,F1,fit_time,predict_time
1,BOW w/ stopwords,LogisticRegression,0.752628,16.151746,0.015629
4,BOW w/o stopwords,LogisticRegression,0.771296,25.979635,0.007005
7,N_gram,LogisticRegression,0.775353,173.404314,0.023020
8,N_gram,Perceptron,0.764052,1.261852,0.016026
10,N_gram_upsampled,LogisticRegression,0.795726,258.851020,0.015625
11,N_gram_upsampled,Perceptron,0.770785,1.929641,0.016026
15,TF_IDF_upsampled,LogisticRegression,0.750458,6.327015,0.015623


**Финальный вывод**

1. Лучшие результы как метрики, так и времени обучения показывает модель логической регрессии;
2. **Максимальный** достижимый результат метрики F1 равный **0.796** был достигнут при использовании *LogisticRegression* с использованием мешка слова с N-граммами (в данном случае уни- и биграммы) на сбалансированный методом upsampling выборке;
3. Самый быстрый, из проходящих по условию ${F1\geq0.75}$ - самая старая из моделей, перцептрон, с метрикой F1 равной **0.77** (с применением n-грамм на наборе данных с upsampling), и временем обучения в 1.9 секунды.
4. При балансировании классов, метод TF-IDF также начинает показывать результат равный требуемому **0.75**, при этом обладая хорошим временм работы.